### Percobaan 1: Matching untuk 3 Driver dan Customer
#### Distance & Time(dgn Openrouteservice), Order Count & Rating.
*) Referensi code dari project semester lalu.

In [16]:
import pandas as pd
import pyomo.environ as pyo
# import openrouteservice as ors
import requests as req
from dotenv import load_dotenv
import os

In [17]:
# Melakukan loading data client dan driver dari CSV.
# File dummy dari project semester lalu.

clients = pd.read_csv('../datasets/data_client_small.csv')
drivers = pd.read_csv('../datasets/data_driver_small.csv')

In [18]:
clients_data = clients.to_dict('records')
drivers_data = drivers.to_dict('records')
clients_data, drivers_data

([{'id': 20,
   'latitude': -7.250317999983876,
   'longitude': 112.68898399990732,
   'rating_client': 5},
  {'id': 21,
   'latitude': -7.349470999997735,
   'longitude': 112.69903299988196,
   'rating_client': 1},
  {'id': 22,
   'latitude': -7.312413999992556,
   'longitude': 112.76892399970548,
   'rating_client': 4},
  {'id': 23,
   'latitude': -7.332430999995354,
   'longitude': 112.67340899994664,
   'rating_client': 4},
  {'id': 24,
   'latitude': -7.268699999986445,
   'longitude': 112.65469599999388,
   'rating_client': 2},
  {'id': 25,
   'latitude': -7.283033999988449,
   'longitude': 112.7348569997915,
   'rating_client': 5}],
 [{'id': 10,
   'latitude': -7.301529999991034,
   'longitude': 112.78152199967369,
   'rating_driver': 3,
   'order_count': 78,
   'total_trip': 342.73199999997945,
   'time_idle': 683},
  {'id': 11,
   'latitude': -7.257838999984927,
   'longitude': 112.79037999965132,
   'rating_driver': 4,
   'order_count': 48,
   'total_trip': 473.2319999999585,

In [38]:
# Load env untuk store API key
load_dotenv(override=True)

# Memo untuk tidak memanggil API untuk data yang sama.
memo = {}

In [41]:
# Function untuk request ke ORS
def create_matrix(data, locations):
    durations_matrix, distances_matrix = {}, {}
    for i, src in enumerate(locations):
        durations_matrix[src] = {}
        distances_matrix[src] = {}
        for j, dest in enumerate(locations):
            durations_matrix[src][dest] = data['durations'][i][j]
            distances_matrix[src][dest] = data['distances'][i][j]

    return distances_matrix, durations_matrix

def get_location_info(locations):
    data = memo.get(tuple(locations))

    if data != None:
        print("Cache hit")
        result = create_matrix(data, locations)
        memo[tuple(locations)] = data
        return result

    key = os.getenv('OPENROUTESERVICE_KEY')
    headers = {
        'Authorization': key,
        'Content-Type': 'application/json; charset=utf-8',
        'Accept': 'application/json, application/geo+json',
    }
    body = {
        'locations': [[i for i in locs] for locs in locations],
        'metrics': ['distance', 'duration'],
    }

    res = req.post(
        'https://api.openrouteservice.org/v2/matrix/driving-car',
        json=body,
        headers=headers
    )
    
    print(res.status_code)
    data = res.json()
    memo[tuple(locations)] = data
    return create_matrix(data, locations)

locations = []
for i in drivers_data:
    locations.append((i['longitude'], i['latitude']))
for i in clients_data:
    locations.append((i['longitude'], i['latitude']))

i = 0
while (i <= 10):
    try:
        distances, durations = get_location_info(locations)
        break
    except Exception as e:
        i += 1
        continue

In [21]:
# Definisikan weight.
# Weight pada kondisi positif dan negatif, negatif berarti cenderung untuk lebih dipilih karena
#  menggunakan metode Minimize pada rumus.
weights = {
    'distance'      : { 'target': 0, 'positive': 1, 'negative': -1 },
    'duration'      : { 'target': 0, 'positive': 1, 'negative': -1 },
    # 'order_count'   : { 'target': 0, 'positive': 1, 'negative': -1 },
    'rating'        : { 'target': 0, 'positive': -1, 'negative': 1 },
}

In [22]:
# Karena data yang berpengaruh hanya data yang digabungkan antara keduanya, 
#   maka kami mengumpulkan data setelah dirumuskan dari masing-masing pasangan penumpang-driver
data: dict[tuple, dict[str, any]] = {}
for d in drivers_data:
    d_id = d['id']
    data[d_id] = {}
    for c in clients_data:
        c_id = c['id']
        distance = distances[(d['longitude'], d['latitude'])][(c['longitude'], c['latitude'])]
        duration = durations[(d['longitude'], d['latitude'])][(c['longitude'], c['latitude'])]
        # rating = abs(d['rating'] - c['rating'])

        data[d_id][c_id] = {
            "distance": distance,
            "duration": duration,
            # "rating": rating,
        }

data

{10: {20: {'distance': 22617.7, 'duration': 1600.23},
  21: {'distance': 14591.75, 'duration': 942.83},
  22: {'distance': 2917.21, 'duration': 330.92},
  23: {'distance': 16769.77, 'duration': 1440.84},
  24: {'distance': 19956.05, 'duration': 1393.99},
  25: {'distance': 9128.12, 'duration': 565.12}},
 11: {20: {'distance': 20800.08, 'duration': 1595.44},
  21: {'distance': 22680.22, 'duration': 1458.84},
  22: {'distance': 9402.35, 'duration': 953.1},
  23: {'distance': 24858.24, 'duration': 1956.85},
  24: {'distance': 22896.41, 'duration': 1477.92},
  25: {'distance': 11969.56, 'duration': 904.96}},
 12: {20: {'distance': 31885.53, 'duration': 2038.61},
  21: {'distance': 14446.79, 'duration': 977.34},
  22: {'distance': 7163.54, 'duration': 623.95},
  23: {'distance': 22386.64, 'duration': 1581.95},
  24: {'distance': 29223.88, 'duration': 1832.37},
  25: {'distance': 18395.95, 'duration': 1003.51}},
 13: {20: {'distance': 8382.64, 'duration': 989.06},
  21: {'distance': 20733.02

In [34]:
# Model pyomo utk solve GP
model = pyo.ConcreteModel()
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

# Variabel yang digunakan untuk mengidentifikasi driver dan customer adalah ID.
drivers = [d['id'] for d in drivers_data]
clients = [c['id'] for c in clients_data]

# Mendaftarkan semua kombinasi antara ID driver dan client.
model.x = pyo.Var(drivers, clients, domain=pyo.NonNegativeIntegers)

# Rumus untuk menghitung penalty negatif dan positif
def get_penalty(target, val, positive=True):
  if positive:
    if val > target: return val - target
    else: return 0
  else:
    if val < target: return target - val
    else: return 0

# Penalty = (weight positive * X * deviasi dgn target apabila >) + (weight negatif * X * deviasi dgn target apabila <)
# Salah satu dari deviasi ini akan menjadi 0 (target: 3, val: 5 maka sisi negatif akan 0)
def penalty(c, d, param):
  m = weights[param]
  return (
    (m['positive'] * model.x[d, c] * get_penalty(m['target'], data[d][c][param], True)) + 
    (m['negative'] * model.x[d, c] * get_penalty(m['target'], data[d][c][param], False))
  )

# Hitung penalti dari matching yang sesuai (DV jadi 1.0)
# Ini dipakai di paling terakhir untuk menampilkan perhitungan penalti.
def v_penalty(c, d, param):
  m = weights[param]
  return (
    (m['positive'] * 1 * get_penalty(m['target'], data[d][c][param], True)) + 
    (m['negative'] * 1 * get_penalty(m['target'], data[d][c][param], False))
  )

# Objective function: minimalkan jumlah dari semua penalty masing-masing kolom data gabungan.
model.Cost = pyo.Objective(
    expr = sum([
        penalty(c, d, 'distance') + 
        penalty(c, d, 'duration')
        for c in clients for d in drivers
        # penalty(c, d, 'order_count') + 
        # penalty(c, d, 'rating') 
      ]),
    sense = pyo.minimize)

# Constraint yang digunakan yaitu pada jumlah nilai variabel pada setiap driver dengan semua kombinasi pasangan penumpangnya antara 0 atau 1 karena hanya memperbolehkan 1 driver mengambil 1 customer atau tidak mengambil customer sama sekali.
model.driver = pyo.ConstraintList()
for d in drivers:
  model.driver.add(sum(model.x[d, c] for c in clients) == 1)

model.client = pyo.ConstraintList()
for c in clients:
  model.client.add(sum(model.x[d, c] for d in drivers) == 1)

# Menambahkan constraint banned untuk pasangan penumpang dan driver yang tidak diperbolehkan ada di hasil matching.
# model.banned = pyo.Constraint(expr=model.x[11, 20] == 0);

In [35]:
# Gunakan solver utk solve
result = pyo.SolverFactory('cbc').solve(model)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 59940.13
  Upper bound: 59940.13
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 36
  Number of nonzeros: 36
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of cre

In [36]:
# Menampilkan hasil matching dalam bentuk table (dari bawaan DataFrame)

print("Hasil dari Assignment Matching\n")
df = pd.DataFrame([], columns=clients, index=drivers)

for c in clients:
  mdr = []
  for d in drivers:
    mdr.append(model.x[d,c]())
  df[c] = mdr

df

Hasil dari Assignment Matching



,20,21,22,23,24,25
10,0.0,0.0,1.0,0.0,0.0,0.0
11,1.0,0.0,0.0,0.0,0.0,0.0
12,0.0,1.0,0.0,0.0,0.0,0.0
13,0.0,0.0,0.0,0.0,1.0,0.0
14,0.0,0.0,0.0,0.0,0.0,1.0
15,0.0,0.0,0.0,1.0,0.0,0.0


In [37]:
# Menampilkan hasil matching dan list penalti.
from pprint import pprint

result_dict = df.to_dict('dict')

for i in result_dict:
        for j in result_dict[i]:
                s = result_dict[i][j]
                if s == 1.0:
                        print(f"{j} ==> {i}")
print()

def find_pen(i):
        for j in clients:
                print(f"---- Penalties for Driver {i} -> Client {j}: ")
                distance_pen = v_penalty(j, i, 'distance')
                duration_pen = v_penalty(j, i, 'duration')
                # rating_pen = v_penalty(j, i, 'rating')
                # order_count_pen = v_penalty(j, i, 'order_count')
                print("\tDistance:", distance_pen)
                print("\tDuration:", duration_pen)
                # print("\tRating:", rating_pen)
                # print("\tOrder count:", order_count_pen)
                print("\tTotal penalty:", distance_pen + duration_pen)

for i in drivers:
        print(f"Driver {i}:")
        find_pen(i)
        for j in clients:
                if result_dict[j][i] == 1:
                        print(f"Match: Driver {i} -> Client {j}")
        print()


11 ==> 20
12 ==> 21
10 ==> 22
15 ==> 23
13 ==> 24
14 ==> 25

Driver 10:
---- Penalties for Driver 10 -> Client 20: 
	Distance: 22617.7
	Duration: 1600.23
	Total penalty: 24217.93
---- Penalties for Driver 10 -> Client 21: 
	Distance: 14591.75
	Duration: 942.83
	Total penalty: 15534.58
---- Penalties for Driver 10 -> Client 22: 
	Distance: 2917.21
	Duration: 330.92
	Total penalty: 3248.13
---- Penalties for Driver 10 -> Client 23: 
	Distance: 16769.77
	Duration: 1440.84
	Total penalty: 18210.61
---- Penalties for Driver 10 -> Client 24: 
	Distance: 19956.05
	Duration: 1393.99
	Total penalty: 21350.04
---- Penalties for Driver 10 -> Client 25: 
	Distance: 9128.12
	Duration: 565.12
	Total penalty: 9693.240000000002
Match: Driver 10 -> Client 22

Driver 11:
---- Penalties for Driver 11 -> Client 20: 
	Distance: 20800.08
	Duration: 1595.44
	Total penalty: 22395.52
---- Penalties for Driver 11 -> Client 21: 
	Distance: 22680.22
	Duration: 1458.84
	Total penalty: 24139.06
---- Penalties for D